## Lesson: Toy Differential Privacy - Simple Database Queries

In this section we're going to play around with Differential Privacy in the context of a database query. The database is going to be a VERY simple database with only one boolean column. Each row corresponds to a person. Each value corresponds to whether or not that person has a certain private attribute (such as whether they have a certain disease, or whether they are above/below a certain age). We are then going to learn how to know whether a database query over such a small database is differentially private or not - and more importantly - what techniques are at our disposal to ensure various levels of privacy


### First We Create a Simple Database

Step one is to create our database - we're going to do this by initializing a random list of 1s and 0s (which are the entries in our database). Note - the number of entries directly corresponds to the number of people in our database.

In [1]:
import torch

# the number of entries in our database
num_entries = 5000

db = torch.rand(num_entries) > 0.5
db

tensor([0, 1, 1,  ..., 0, 0, 1], dtype=torch.uint8)

## Project: Generate Parallel Databases

Key to the definition of differenital privacy is the ability to ask the question "When querying a database, if I removed someone from the database, would the output of the query be any different?". Thus, in order to check this, we must construct what we term "parallel databases" which are simply databases with one entry removed. 

In this first project, I want you to create a list of every parallel database to the one currently contained in the "db" variable. Then, I want you to create a function which both:

- creates the initial database (db)
- creates all parallel databases

In [95]:
def mk_dbs(num_entries: int, p=0.5):
    db = torch.rand(num_entries) > (1-p)
    dbs = [torch.cat((db[0:i], db[i+1:])) for i in range(len(db))]
    return db, dbs

In [3]:
db, dbs = mk_dbs(5000)

# Lesson: Towards Evaluating The Differential Privacy of a Function

Intuitively, we want to be able to query our database and evaluate whether or not the result of the query is leaking "private" information. As mentioned previously, this is about evaluating whether the output of a query changes when we remove someone from the database. Specifically, we want to evaluate the *maximum* amount the query changes when someone is removed (maximum over all possible people who could be removed). So, in order to evaluate how much privacy is leaked, we're going to iterate over each person in the database and measure the difference in the output of the query relative to when we query the entire database. 

Just for the sake of argument, let's make our first "database query" a simple sum. Aka, we're going to count the number of 1s in the database.

In [4]:
db, pdbs = mk_dbs(5000)

In [5]:
def query(db):
    return db.sum()

In [6]:
full_db_result = query(db)

In [7]:
sensitivity = 0
for pdb in pdbs:
    pdb_result = query(pdb)
    
    db_distance = torch.abs(pdb_result - full_db_result)
    
    if(db_distance > sensitivity):
        sensitivity = db_distance

In [8]:
sensitivity

tensor(1)

# Project - Evaluating the Privacy of a Function

In the last section, we measured the difference between each parallel db's query result and the query result for the entire database and then calculated the max value (which was 1). This value is called "sensitivity", and it corresponds to the function we chose for the query. Namely, the "sum" query will always have a sensitivity of exactly 1. However, we can also calculate sensitivity for other functions as well.

Let's try to calculate sensitivity for the "mean" function.

In [9]:
def sensitivity(query, num_entries: int) -> torch.Tensor:
    db, pdbs = mk_dbs(num_entries)
    full_db_result = query(db)
    return max([torch.abs(query(pdb) - full_db_result) for pdb in pdbs])

In [10]:
sensitivity(lambda db: db.max().type(torch.FloatTensor), 5)

tensor(0.)

Wow! That sensitivity is WAY lower. Note the intuition here. "Sensitivity" is measuring how sensitive the output of the query is to a person being removed from the database. For a simple sum, this is always 1, but for the mean, removing a person is going to change the result of the query by rougly 1 divided by the size of the database (which is much smaller). Thus, "mean" is a VASTLY less "sensitive" function (query) than SUM.

# Project: Calculate L1 Sensitivity For Threshold

In this first project, I want you to calculate the sensitivty for the "threshold" function. 

- First compute the sum over the database (i.e. sum(db)) and return whether that sum is greater than a certain threshold.
- Then, I want you to create databases of size 10 and threshold of 5 and calculate the sensitivity of the function. 
- Finally, re-initialize the database 10 times and calculate the sensitivity each time.

In [11]:
def threshold_query(threshold):
    return lambda db: (db.sum() > threshold).float()

In [12]:
sensitivity(threshold_query(5), 10)

tensor(0.)

In [13]:
for i in range(10):
    print(sensitivity(threshold_query(5), 10))

tensor(0.)
tensor(0.)
tensor(1.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)


# Lesson: A Basic Differencing Attack

Sadly none of the functions we've looked at so far are differentially private (despite them having varying levels of sensitivity). The most basic type of attack can be done as follows.

Let's say we wanted to figure out a specific person's value in the database. All we would have to do is query for the sum of the entire database and then the sum of the entire database without that person!

# Project: Perform a Differencing Attack on Row 10

In this project, I want you to construct a database and then demonstrate how you can use two different sum queries to explose the value of the person represented by row 10 in the database (note, you'll need to use a database with at least 10 rows)

In [14]:
db, _ = mk_dbs(100)
def query(db): return db.sum()
query(db) - query(torch.cat((db[:9], db[10:])))

tensor(0)

In [15]:
db[9]

tensor(0, dtype=torch.uint8)

# Project: Local Differential Privacy

As you can see, the basic sum query is not differentially private at all! In truth, differential privacy always requires a form of randomness added to the query. Let me show you what I mean.

### Randomized Response (Local Differential Privacy)

Let's say I have a group of people I wish to survey about a very taboo behavior which I think they will lie about (say, I want to know if they have ever committed a certain kind of crime). I'm not a policeman, I'm just trying to collect statistics to understand the higher level trend in society. So, how do we do this? One technique is to add randomness to each person's response by giving each person the following instructions (assuming I'm asking a simple yes/no question):

- Flip a coin 2 times.
- If the first coin flip is heads, answer honestly
- If the first coin flip is tails, answer according to the second coin flip (heads for yes, tails for no)!

Thus, each person is now protected with "plausible deniability". If they answer "Yes" to the question "have you committed X crime?", then it might becasue they actually did, or it might be becasue they are answering according to a random coin flip. Each person has a high degree of protection. Furthermore, we can recover the underlying statistics with some accuracy, as the "true statistics" are simply averaged with a 50% probability. Thus, if we collect a bunch of samples and it turns out that 60% of people answer yes, then we know that the TRUE distribution is actually centered around 70%, because 70% averaged wtih 50% (a coin flip) is 60% which is the result we obtained. 

However, it should be noted that, especially when we only have a few samples, the this comes at the cost of accuracy. This tradeoff exists across all of Differential Privacy. The greater the privacy protection (plausible deniability) the less accurate the results. 

Let's implement this local DP for our database before!

In [51]:
import numpy as np
rng = np.random.RandomState(2)

def add_noise(db):
    return torch.tensor([act if det == 1 else mask
            for act, det, mask in zip(db, rng.binomial(1, 0.5, len(db)), rng.binomial(1, 0.5, len(db)))])

In [64]:
def test_size(n: int):
    db, _ = mk_dbs(n)
    dbn = add_noise(db)
    print(" original: {:0.4f}".format(db.float().mean()))
    print("    noisy: {:0.4f}".format(dbn.float().mean()))
    print("corrected: {:0.4f}".format(dbn.float().mean()*2 - 0.5))

In [65]:
test_size(10)

 original: 0.7000
    noisy: 0.7000
corrected: 0.9000


In [66]:
test_size(100)

 original: 0.5000
    noisy: 0.5600
corrected: 0.6200


In [67]:
test_size(1000)

 original: 0.5120
    noisy: 0.4900
corrected: 0.4800


In [68]:
test_size(10000)

 original: 0.5063
    noisy: 0.5007
corrected: 0.5014


# Project: Varying Amounts of Noise

In this project, I want you to augment the randomized response query (the one we just wrote) to allow for varying amounts of randomness to be added. Specifically, I want you to bias the coin flip to be higher or lower and then run the same experiment. 

Note - this one is a bit tricker than you might expect. You need to both adjust the likelihood of the first coin flip AND the de-skewing at the end (where we create the "augmented_result" variable).

In [96]:
def add_noise_p(db: torch.Tensor, p: float) -> torch.Tensor:
    dets = (torch.rand(db.shape) > p).float()
    masks = (torch.rand(db.shape) > 0.5).float()
    return db * dets + (1-dets) * masks

def test_size_p(n, p):
    db, _ = mk_dbs(n, p=0.1)
    db = db.float()
    dbn = add_noise_p(db, p).float()
    print(f" original: {db.mean():0.4f}")
    print(f"    noisy: {dbn.mean():0.4f}")
    print(f"corrected: {(dbn.mean() - p*0.5) / (1-p):0.4f}")

In [97]:
test_size_p(10, 0.5)

 original: 0.0000
    noisy: 0.4000
corrected: 0.3000


In [98]:
test_size_p(1000, 0.5)

 original: 0.1060
    noisy: 0.3060
corrected: 0.1120


In [100]:
test_size_p(1000, 0.75)

 original: 0.0980
    noisy: 0.3840
corrected: 0.0360


In [101]:
test_size_p(1000, 0.25)

 original: 0.1110
    noisy: 0.2080
corrected: 0.1107


# Lesson: The Formal Definition of Differential Privacy

The previous method of adding noise was called "Local Differentail Privacy" because we added noise to each datapoint individually. This is necessary for some situations wherein the data is SO sensitive that individuals do not trust noise to be added later. However, it comes at a very high cost in terms of accuracy. 

However, alternatively we can add noise AFTER data has been aggregated by a function. This kind of noise can allow for similar levels of protection with a lower affect on accuracy. However, participants must be able to trust that no-one looked at their datapoints _before_ the aggregation took place. In some situations this works out well, in others (such as an individual hand-surveying a group of people), this is less realistic.

Nevertheless, global differential privacy is incredibly important because it allows us to perform differential privacy on smaller groups of individuals with lower amounts of noise. Let's revisit our sum functions.

In [40]:
db, pdbs = create_db_and_parallels(100)

def query(db):
    return torch.sum(db.float())

def M(db):
    query(db) + noise

query(db)

tensor(40.)

So the idea here is that we want to add noise to the output of our function. We actually have two different kinds of noise we can add - Laplacian Noise or Gaussian Noise. However, before we do so at this point we need to dive into the formal definition of Differential Privacy.

![alt text](dp_formula.png "Title")

_Image From: "The Algorithmic Foundations of Differential Privacy" - Cynthia Dwork and Aaron Roth - https://www.cis.upenn.edu/~aaroth/Papers/privacybook.pdf_

This definition does not _create_ differential privacy, instead it is a measure of how much privacy is afforded by a query M. Specifically, it's a comparison between running the query M on a database (x) and a parallel database (y). As you remember, parallel databases are defined to be the same as a full database (x) with one entry/person removed.

Thus, this definition says that FOR ALL parallel databases, the maximum distance between a query on database (x) and the same query on database (y) will be e^epsilon, but that occasionally this constraint won't hold with probability delta. Thus, this theorem is called "epsilon delta" differential privacy.

# Epsilon

Let's unpack the intuition of this for a moment. 

Epsilon Zero: If a query satisfied this inequality where epsilon was set to 0, then that would mean that the query for all parallel databases outputed the exact same value as the full database. As you may remember, when we calculated the "threshold" function, often the Sensitivity was 0. In that case, the epsilon also happened to be zero.

Epsilon One: If a query satisfied this inequality with epsilon 1, then the maximum distance between all queries would be 1 - or more precisely - the maximum distance between the two random distributions M(x) and M(y) is 1 (because all these queries have some amount of randomness in them, just like we observed in the last section).

# Delta

Delta is basically the probability that epsilon breaks. Namely, sometimes the epsilon is different for some queries than it is for others. For example, you may remember when we were calculating the sensitivity of threshold, most of the time sensitivity was 0 but sometimes it was 1. Thus, we could calculate this as "epsilon zero but non-zero delta" which would say that epsilon is perfect except for some probability of the time when it's arbitrarily higher. Note that this expression doesn't represent the full tradeoff between epsilon and delta.

# Lesson: How To Add Noise for Global Differential Privacy

In this lesson, we're going to learn about how to take a query and add varying amounts of noise so that it satisfies a certain degree of differential privacy. In particular, we're going to leave behind the Local Differential privacy previously discussed and instead opt to focus on Global differential privacy. 

So, to sum up, this lesson is about adding noise to the output of our query so that it satisfies a certain epsilon-delta differential privacy threshold.

There are two kinds of noise we can add - Gaussian Noise or Laplacian Noise. Generally speaking Laplacian is better, but both are still valid. Now to the hard question...

### How much noise should we add?

The amount of noise necessary to add to the output of a query is a function of four things:

- the type of noise (Gaussian/Laplacian)
- the sensitivity of the query/function
- the desired epsilon (ε)
- the desired delta (δ)

Thus, for each type of noise we're adding, we have different way of calculating how much to add as a function of sensitivity, epsilon, and delta. We're going to focus on Laplacian noise. Laplacian noise is increased/decreased according to a "scale" parameter b. We choose "b" based on the following formula.

b = sensitivity(query) / epsilon

In other words, if we set b to be this value, then we know that we will have a privacy leakage of <= epsilon. Furthermore, the nice thing about Laplace is that it guarantees this with delta == 0. There are some tunings where we can have very low epsilon where delta is non-zero, but we'll ignore them for now.

### Querying Repeatedly

- if we query the database multiple times - we can simply add the epsilons (Even if we change the amount of noise and their epsilons are not the same).

# Project: Create a Differentially Private Query

In this project, I want you to take what you learned in the previous lesson and create a query function which sums over the database and adds just the right amount of noise such that it satisfies an epsilon constraint. Write a query for both "sum" and for "mean". Ensure that you use the correct sensitivity measures for both.

In [107]:
# try this project here!
def sum_query(db, epsilon=1.0):
    sum_sensitivity = 1.0
    return db.float().sum() + np.random.laplace(loc=0, scale=sum_sensitivity/epsilon)

In [145]:
db, _ = mk_dbs(1000)
print(db.float().sum())
sum_query(db, epsilon=0.01)

tensor(484.)


tensor(399.2141)

In [146]:
def mean_query(db, epsilon=1.0):
    mean_sensitivity = 1/len(db)
    return db.float().mean() + np.random.laplace(loc=0, scale=mean_sensitivity/epsilon)

In [154]:
db, _ = mk_dbs(1000)
print(db.float().mean())
mean_query(db, epsilon=0.1)

tensor(0.5090)


tensor(0.5040)

# Lesson: Differential Privacy for Deep Learning

So in the last lessons you may have been wondering - what does all of this have to do with Deep Learning? Well, these same techniques we were just studying form the core primitives for how Differential Privacy provides guarantees in the context of Deep Learning. 

Previously, we defined perfect privacy as "a query to a database returns the same value even if we remove any person from the database", and used this intuition in the description of epsilon/delta. In the context of deep learning we have a similar standard.

Training a model on a dataset should return the same model even if we remove any person from the dataset.

Thus, we've replaced "querying a database" with "training a model on a dataset". In essence, the training process is a kind of query. However, one should note that this adds two points of complexity which database queries did not have:

    1. do we always know where "people" are referenced in the dataset?
    2. neural models rarely never train to the same output model, even on identical data

The answer to (1) is to treat each training example as a single, separate person. Strictly speaking, this is often overly zealous as some training examples have no relevance to people and others may have multiple/partial (consider an image with multiple people contained within it). Thus, localizing exactly where "people" are referenced, and thus how much your model would change if people were removed, is challenging.

The answer to (2) is also an open problem - but several interesitng proposals have been made. We're going to focus on one of the most popular proposals, PATE.

## An Example Scenario: A Health Neural Network

First we're going to consider a scenario - you work for a hospital and you have a large collection of images about your patients. However, you don't know what's in them. You would like to use these images to develop a neural network which can automatically classify them, however since your images aren't labeled, they aren't sufficient to train a classifier. 

However, being a cunning strategist, you realize that you can reach out to 10 partner hospitals which DO have annotated data. It is your hope to train your new classifier on their datasets so that you can automatically label your own. While these hospitals are interested in helping, they have privacy concerns regarding information about their patients. Thus, you will use the following technique to train a classifier which protects the privacy of patients in the other hospitals.

- 1) You'll ask each of the 10 hospitals to train a model on their own datasets (All of which have the same kinds of labels)
- 2) You'll then use each of the 10 partner models to predict on your local dataset, generating 10 labels for each of your datapoints
- 3) Then, for each local data point (now with 10 labels), you will perform a DP query to generate the final true label. This query is a "max" function, where "max" is the most frequent label across the 10 labels. We will need to add laplacian noise to make this Differentially Private to a certain epsilon/delta constraint.
- 4) Finally, we will retrain a new model on our local dataset which now has labels. This will be our final "DP" model.

So, let's walk through these steps. I will assume you're already familiar with how to train/predict a deep neural network, so we'll skip steps 1 and 2 and work with example data. We'll focus instead on step 3, namely how to perform the DP query for each example using toy data.

So, let's say we have 10,000 training examples, and we've got 10 labels for each example (from our 10 "teacher models" which were trained directly on private data). Each label is chosen from a set of 10 possible labels (categories) for each image.

In [2]:
import numpy as np

In [3]:
num_teachers = 10 # we're working with 10 partner hospitals
num_examples = 10000 # the size of OUR dataset
num_labels = 10 # number of lablels for our classifier

In [4]:
preds = (np.random.rand(num_teachers, num_examples) * num_labels).astype(int).transpose(1,0) # fake predictions

In [5]:
new_labels = list()
for an_image in preds:

    label_counts = np.bincount(an_image, minlength=num_labels)

    epsilon = 0.1
    beta = 1 / epsilon

    for i in range(len(label_counts)):
        label_counts[i] += np.random.laplace(0, beta, 1)

    new_label = np.argmax(label_counts)
    
    new_labels.append(new_label)

In [57]:
# new_labels

# PATE Analysis

In [6]:
labels = np.array([9, 9, 3, 6, 9, 9, 9, 9, 8, 2])
counts = np.bincount(labels, minlength=10)
query_result = np.argmax(counts)
query_result

9

In [7]:
from syft.frameworks.torch.differential_privacy import pate

In [11]:
num_teachers, num_examples, num_labels = (100, 100, 10)
preds = (np.random.rand(num_teachers, num_examples) * num_labels).astype(int) #fake preds
indices = (np.random.rand(num_examples) * num_labels).astype(int) # true answers

preds[:,0:10] *= 0

data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=indices, noise_eps=0.1, delta=1e-5)

assert data_dep_eps < data_ind_eps



In [12]:
data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=indices, noise_eps=0.1, delta=1e-5)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

Data Independent Epsilon: 11.756462732485115
Data Dependent Epsilon: 1.52655213289881


In [13]:
preds[:,0:50] *= 0

In [14]:
data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=indices, noise_eps=0.1, delta=1e-5, moments=20)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

Data Independent Epsilon: 11.756462732485115
Data Dependent Epsilon: 0.9029013677789843


# Where to Go From Here


Read:
    - Algorithmic Foundations of Differential Privacy: https://www.cis.upenn.edu/~aaroth/Papers/privacybook.pdf
    - Deep Learning with Differential Privacy: https://arxiv.org/pdf/1607.00133.pdf
    - The Ethical Algorithm: https://www.amazon.com/Ethical-Algorithm-Science-Socially-Design/dp/0190948205
   
Topics:
    - The Exponential Mechanism
    - The Moment's Accountant
    - Differentially Private Stochastic Gradient Descent

Advice:
    - For deployments - stick with public frameworks!
    - Join the Differential Privacy Community
    - Don't get ahead of yourself - DP is still in the early days

# Section Project:

For the final project for this section, you're going to train a DP model using this PATE method on the MNIST dataset, provided below.

In [72]:
import torchvision.datasets as datasets
from torchvision import transforms

transform = transforms.Compose([transforms.Grayscale(3),
                                transforms.ToTensor(),
                                transforms.Normalize((0.485, 0.456, 0.406),
                                                     (0.229, 0.224, 0.225))])

mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

Model architecture:

In [60]:
from collections import namedtuple

import torch
from torch import nn
from torch import optim
from torchvision import models

Harness = namedtuple('Model', ['model', 'criterion', 'optimizer'])

n_classes = len(set(mnist_trainset.targets.tolist()))

def mk_model() -> Harness:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = models.densenet121(pretrained=True)

    for param in model.parameters():
        param.requires_grad = False

    model.classifier = nn.Sequential(nn.Linear(1024, 256),
                                     nn.ReLU(),
                                     nn.Dropout(0.2),
                                     nn.Linear(256, n_classes),
                                     nn.LogSoftmax(dim=1))
    criterion = nn.NLLLoss()
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)
    model.to(device)
    return Harness(model, criterion, optimizer)

Partition the training set:

In [6]:
from torch.utils.data import DataLoader, Subset
n_teachers = 10

teacher_assignment = (torch.rand(60000) * n_teachers).int()

teacher_trainset = []

for teacher_idx in range(n_teachers):
    example_indices = [ix for ix, t in enumerate(teacher_assignment) if t == teacher_idx]
    teacher_trainset.append(DataLoader(Subset(mnist_trainset, example_indices), batch_size=64, shuffle=True))

Create teacher models:

In [7]:
teachers = [mk_model() for _ in range(n_teachers)]

The training function:

In [8]:
def train(harness: Harness, train_data, test_data=None, epochs=3, print_every=20):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    train_loss = 0
    steps = 0
    for epoch in range(epochs):
        for inputs, labels in train_data:
            steps += 1

            inputs, labels = inputs.to(device), labels.to(device)

            harness.optimizer.zero_grad()

            logps = harness.model.forward(inputs)
            loss = harness.criterion(logps, labels)
            loss.backward()
            harness.optimizer.step()

            train_loss += loss.item()

            if steps % print_every == 0:
                if test_data is not None:
                    test_loss = 0
                    accuracy = 0
                    harness.model.eval()
                    with torch.no_grad():
                        for inputs, labels in test_data:
                            inputs, labels = inputs.to(device), labels.to(device)
                            logps = harness.model.forward(inputs)
                            batch_loss = harness.criterion(logps, labels)

                            test_loss += batch_loss.item()

                            ps = torch.exp(logps)
                            top_p, top_class = ps.topk(1, dim=1)
                            equals = top_class == labels.view(*top_class.shape)
                            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    print(f"Epoch {epoch+1}/{epochs} "
                          f"Train loss: {train_loss/print_every:.3f} "
                          f"Test loss: {test_loss/len(test_data):.3f} "
                          f"Test accuracy: {accuracy/len(test_data):.3f}")
                    harness.model.train()
                else:
                    print(f"Epoch {epoch+1}/{epochs} "
                          f"Train loss: {train_loss/print_every:.3f} ")
                train_loss = 0

Train all teachers:

In [9]:
for i in range(n_teachers):
    print(f"Training teacher {i+1} =============================")
    train(teachers[i], teacher_trainset[i])

Training teacher 1 =============================
Epoch 1/3 Train loss: 1.909 
Epoch 1/3 Train loss: 1.450 
Epoch 1/3 Train loss: 1.327 
Epoch 1/3 Train loss: 1.298 
Epoch 2/3 Train loss: 1.080 
Epoch 2/3 Train loss: 1.037 
Epoch 2/3 Train loss: 1.017 
Epoch 2/3 Train loss: 1.063 
Epoch 2/3 Train loss: 1.058 
Epoch 3/3 Train loss: 0.909 
Epoch 3/3 Train loss: 0.910 
Epoch 3/3 Train loss: 0.914 
Epoch 3/3 Train loss: 0.931 
Training teacher 2 =============================
Epoch 1/3 Train loss: 1.974 
Epoch 1/3 Train loss: 1.533 
Epoch 1/3 Train loss: 1.274 
Epoch 1/3 Train loss: 1.255 
Epoch 2/3 Train loss: 1.180 
Epoch 2/3 Train loss: 1.020 
Epoch 2/3 Train loss: 1.036 
Epoch 2/3 Train loss: 0.965 
Epoch 2/3 Train loss: 0.986 
Epoch 3/3 Train loss: 0.975 
Epoch 3/3 Train loss: 0.956 
Epoch 3/3 Train loss: 0.898 
Epoch 3/3 Train loss: 0.919 
Training teacher 3 =============================
Epoch 1/3 Train loss: 1.954 
Epoch 1/3 Train loss: 1.461 
Epoch 1/3 Train loss: 1.282 
Epoch 1/3 Tr

Generate teacher predictions:

In [50]:
from tqdm import tqdm
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for t in range(n_teachers):
    teachers[t].model.eval()

batch_size = 100

teacher_preds = torch.tensor([]).reshape(-1, n_teachers).int()
with torch.no_grad():
    for inputs, _ in tqdm(DataLoader(mnist_trainset, batch_size=batch_size, shuffle=False)):
        batch_preds = torch.tensor([]).reshape((batch_size, -1)).int()
        for t in range(n_teachers):
            logps = teachers[t].model.forward(inputs.to(device))
            ps = torch.exp(logps)
            _, top_class = ps.topk(1, dim=1)
            batch_preds = torch.cat((batch_preds, top_class.to('cpu').int()), dim=1)
        teacher_preds = torch.cat((teacher_preds, batch_preds), dim=0)

100%|██████████| 600/600 [03:26<00:00,  2.89it/s]


In [51]:
teacher_preds.shape

torch.Size([60000, 10])

In [52]:
teacher_preds[:10]

tensor([[7, 5, 8, 7, 5, 5, 3, 5, 7, 7],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [9, 9, 9, 9, 9, 9, 9, 9, 9, 9],
        [2, 5, 2, 2, 2, 5, 2, 2, 5, 5],
        [1, 7, 1, 1, 1, 3, 1, 7, 1, 1],
        [3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [4, 4, 4, 4, 4, 4, 4, 4, 4, 4]], dtype=torch.int32)

Create noisy labels from teacher predictions:

In [67]:
import numpy as np
noisy_labels = []
noise_epsilon = 1.0

for i in tqdm(range(len(teacher_preds))):
    class_counts = np.bincount(teacher_preds[i], minlength=n_classes)
    for i in range(len(class_counts)):
        class_counts[i] += np.random.laplace(0, 1/noise_epsilon, 1)
    label = np.argmax(class_counts)
    noisy_labels.append(label)

100%|██████████| 60000/60000 [00:10<00:00, 5782.50it/s]


In [68]:
noisy_labels[:10]

[7, 0, 4, 1, 9, 2, 1, 3, 1, 4]

In [73]:
mnist_trainset.targets[:10]

tensor([5, 0, 4, 1, 9, 2, 1, 3, 1, 4])

What does PATE analysis tell us?

In [92]:
from syft.frameworks.torch.differential_privacy import pate
pate.perform_analysis(teacher_preds=teacher_preds.view(n_teachers,-1).numpy(),
                      indices=mnist_trainset.targets.numpy(),
                      noise_eps=noise_epsilon,
                      moments=2)

(120005.75646273249, 120005.75646273249)

Nothing useful. With our choice of noise epsilon the effective DP epsilon is huge. Can't use more moments because pysyft crashes.

In [93]:
mnist_trainset.targets = torch.tensor(noisy_labels)

In [94]:
mnist_trainset.targets[:10]

tensor([7, 0, 4, 1, 9, 2, 1, 3, 1, 4])

Train a model on noisy labels:

In [96]:
dp = mk_model()
train(dp,
      train_data=DataLoader(mnist_trainset, batch_size=64, shuffle=True),
      test_data=DataLoader(mnist_testset, batch_size=64, shuffle=False))

Epoch 1/3 Train loss: 2.055 Test loss: 2.194 Test accuracy: 0.203
Epoch 1/3 Train loss: 1.467 Test loss: 1.545 Test accuracy: 0.495
Epoch 1/3 Train loss: 1.284 Test loss: 1.216 Test accuracy: 0.635
Epoch 1/3 Train loss: 1.079 Test loss: 1.178 Test accuracy: 0.656
Epoch 1/3 Train loss: 1.098 Test loss: 1.056 Test accuracy: 0.692
Epoch 1/3 Train loss: 0.973 Test loss: 1.078 Test accuracy: 0.684
Epoch 1/3 Train loss: 0.987 Test loss: 1.022 Test accuracy: 0.698
Epoch 1/3 Train loss: 0.948 Test loss: 0.969 Test accuracy: 0.712
Epoch 1/3 Train loss: 0.861 Test loss: 0.948 Test accuracy: 0.718
Epoch 1/3 Train loss: 0.895 Test loss: 0.965 Test accuracy: 0.708
Epoch 1/3 Train loss: 0.810 Test loss: 0.956 Test accuracy: 0.720
Epoch 1/3 Train loss: 0.859 Test loss: 0.911 Test accuracy: 0.722
Epoch 1/3 Train loss: 0.917 Test loss: 0.900 Test accuracy: 0.725
Epoch 1/3 Train loss: 0.893 Test loss: 0.869 Test accuracy: 0.732
Epoch 1/3 Train loss: 0.903 Test loss: 0.863 Test accuracy: 0.729
Epoch 1/3 

Epoch 3/3 Train loss: 0.832 Test loss: 0.749 Test accuracy: 0.774
Epoch 3/3 Train loss: 0.720 Test loss: 0.739 Test accuracy: 0.777
Epoch 3/3 Train loss: 0.775 Test loss: 0.780 Test accuracy: 0.768
Epoch 3/3 Train loss: 0.730 Test loss: 0.822 Test accuracy: 0.756
Epoch 3/3 Train loss: 0.714 Test loss: 0.829 Test accuracy: 0.757
Epoch 3/3 Train loss: 0.785 Test loss: 0.783 Test accuracy: 0.757
Epoch 3/3 Train loss: 0.763 Test loss: 0.787 Test accuracy: 0.758
Epoch 3/3 Train loss: 0.694 Test loss: 0.793 Test accuracy: 0.761
Epoch 3/3 Train loss: 0.757 Test loss: 0.801 Test accuracy: 0.765
Epoch 3/3 Train loss: 0.765 Test loss: 0.787 Test accuracy: 0.759
Epoch 3/3 Train loss: 0.717 Test loss: 0.808 Test accuracy: 0.752
Epoch 3/3 Train loss: 0.698 Test loss: 0.819 Test accuracy: 0.765
Epoch 3/3 Train loss: 0.741 Test loss: 0.779 Test accuracy: 0.760
Epoch 3/3 Train loss: 0.725 Test loss: 0.791 Test accuracy: 0.767
Epoch 3/3 Train loss: 0.745 Test loss: 0.756 Test accuracy: 0.768


That's not a great performance, but perhaps (?) it's at least somewhat private...